In [1]:
import os
from owlready2 import *
import pandas as pd
from rdflib import Graph

def convert_to_owl(input_path, output_owl_path): 
    g = Graph()
    input_format = 'ttl' if input_path.endswith('.ttl') else 'xml'
    g.parse(input_path, format=input_format)
    # Serialize the graph to RDF/XML format (commonly used for .owl files)
    g.serialize(destination=output_owl_path, format='pretty-xml')

def count_ontology_elements(ontology_path):
    try:
         
        onto = get_ontology(ontology_path).load()
        
        # Count the elements
        num_classes = len(list(onto.classes()))
        num_individuals = len(list(onto.individuals()))
        num_object_properties = len(list(onto.object_properties()))
        num_data_properties = len(list(onto.data_properties()))
        
        counts = {
            "filename": os.path.basename(ontology_path),
            "classes": num_classes,
            "individuals": num_individuals,
            "object_properties": num_object_properties,
            "data_properties": num_data_properties
        }
        
        return counts
    
    except Exception as e:
        print(f"Error loading {ontology_path}: {e}")
        return None

def count_elements_in_directory(directory_path): 
    ontology_counts = {}
    
    # Supported ontology file extensions
    supported_extensions = (".owl", ".ttl", ".rdf")
    
    # Create a directory for converted files if it doesn't exist
    converted_dir = os.path.join(directory_path, 'converted_files_bkp')
    os.makedirs(converted_dir, exist_ok=True)
     
    for filename in os.listdir(directory_path):
        if filename.endswith(supported_extensions):
            ontology_path = os.path.join(directory_path, filename)
            
            # If the file is .ttl or .rdf, convert it to .owl and move it to 'converted_files_bkp' folder
            if filename.endswith(('.ttl', '.rdf')): 
                converted_path = os.path.join(directory_path, os.path.splitext(filename)[0] + '.owl')
                convert_to_owl(ontology_path, converted_path)
                
                # move ttl / rdf to converted_files_bkp
                new_original_path = os.path.join(converted_dir, filename)
                os.rename(ontology_path, new_original_path)
                
                 
                ontology_path = converted_path
            
            # Count classes/properties in the .owl file
            counts = count_ontology_elements(ontology_path)
            if counts:
                ontology_counts[filename] = counts
    
    return ontology_counts 

In [6]:
directory_path = "../Data"
ontology_counts = count_elements_in_directory(directory_path)

df = pd.DataFrame.from_dict(ontology_counts,orient='index')
df_reset = df.reset_index(drop=True)
df_reset

,filename,classes,individuals,object_properties,data_properties
0,argo.owl,13,8,20,0
1,rdo_1.0.owl,15,0,36,1
2,sio.owl,1584,0,211,1
3,amo.owl,8,0,21,0
4,eco.owl,3337,18,47,3
5,sepio.owl,125,21,210,32
6,EXPO04-19-06.owl,324,0,78,0


In [7]:
print(df_reset.to_string(index=False))

        filename  classes  individuals  object_properties  data_properties
        argo.owl       13            8                 20                0
     rdo_1.0.owl       15            0                 36                1
         sio.owl     1584            0                211                1
         amo.owl        8            0                 21                0
         eco.owl     3337           18                 47                3
       sepio.owl      125           21                210               32
EXPO04-19-06.owl      324            0                 78                0
